In [166]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [259]:
#List of genres from IMDB 

imdb_genres_list = [
    'Action',
    'Adventure',
    'Animation',
    'Biography',
    'Comedy',
    'Crime',
    'Documentary',
    'Drama',
    'Family',
    'Fantasy',
    'Game-Show',
    'History',
    'Horror',
    'Music',
    'Musical',
    'Mystery',
    'News',
    'Reality-TV',
    'Romance',
    'Sci-Fi',
    'Short',
    'Sport',
    'Talk-Show',
    'Thriller',
    'War',
    'Western'
]

In [260]:
titles=[]
entries_wanted = 500  # Pages load in increments of 50

In [261]:
for input_genre in imdb_genres_list:
    start_index = 1
    while start_index < entries_wanted:
        url = f'https://www.imdb.com/search/title/?title_type=tv_series,tv_miniseries&num_votes=5000,&genres={input_genre}&sort=num_votes,desc&start={start_index}&ref_=adv_nxt'
        res = requests.get(url)
        soup = BeautifulSoup(res.text)

        for list_item in soup.find_all('div', attrs={'class':'lister-item mode-advanced'}):
            link = list_item.find_all('a')[1]
            years = list_item.find('span', attrs={'class':'lister-item-year text-muted unbold'})
            description_obj = list_item.find_all('p', attrs={'class':'text-muted'})
            pg_rating = list_item.find('span', attrs={'class':'certificate'})
            genres = list_item.find('span', attrs={'class':'genre'})
            rating = list_item.find('meta', attrs={'itemprop':'ratingValue'})
            num_votes = list_item.find('meta', attrs={'itemprop':'ratingCount'})
            img_thumbnail = list_item.find('img')
            try:
                entry = {
                    'name': link.text,
                    'href': link['href'].strip(),
                    'years': years.text,
                    # 'verification': i.find('h3').text.strip().replace('\n', ' '),
                    # 'input_genre': input_genre,
                    'imdb_description': description_obj[1].text.strip().replace('\n', ' '),
                    'pg_rating': pg_rating.text.strip(),
                    'imdb_genre_tags': genres.text.strip(),
                    'imdb_rating': float(rating['content']),
                    'num_votes': int(num_votes['content']),
                    'img_thumbnail': img_thumbnail['loadlate']
                }
            except:
                pass
            
            titles.append(entry)
        
        start_index += 50


# Technique reference: https://stackoverflow.com/questions/51996139/grabbing-meta-content-with-beautiful-soup

In [262]:
len(titles)

7083

In [263]:
titles_df = pd.DataFrame(titles)
titles_df.drop_duplicates(inplace=True)
titles_df

,name,href,years,imdb_description,pg_rating,imdb_genre_tags,imdb_rating,num_votes,img_thumbnail
0,Game of Thrones,/title/tt0944947/,(2011–2019),Nine noble families fight for control over the...,TV-MA,"Action, Adventure, Drama",9.2,2148311,https://m.media-amazon.com/images/M/MV5BYTRiND...
1,Prison Break,/title/tt0455275/,(2005–2017),"Due to a political conspiracy, an innocent man...",TV-14,"Action, Crime, Drama",8.3,548267,https://m.media-amazon.com/images/M/MV5BMTg3NT...
2,Vikings,/title/tt2306299/,(2013–2020),Vikings transports us to the brutal and myster...,TV-MA,"Action, Adventure, Drama",8.5,547494,https://m.media-amazon.com/images/M/MV5BODk4Zj...
3,The Boys,/title/tt1190634/,(2019– ),A group of vigilantes set out to take down cor...,TV-MA,"Action, Comedy, Crime",8.7,542317,https://m.media-amazon.com/images/M/MV5BOTEyND...
4,The Mandalorian,/title/tt8111088/,(2019– ),The travels of a lone bounty hunter in the out...,TV-14,"Action, Adventure, Fantasy",8.7,527088,https://m.media-amazon.com/images/M/MV5BZjRlZD...
...,...,...,...,...,...,...,...,...,...
7073,Gunsmoke,/title/tt0047736/,(1955–1975),Marshal Matt Dillon keeps the peace in rough-a...,TV-PG,Western,8.1,7810,https://m.media-amazon.com/images/M/MV5BMTcxMD...
7078,Kung Fu,/title/tt0068093/,(1972–1975),The adventures of Shaolin Monk Kwai Chang Cain...,TV-PG,"Adventure, Drama, Western",7.6,6594,https://m.media-amazon.com/images/M/MV5BMjNkMz...
7079,Broken Trail,/title/tt0482857/,(2006),A veteran cowboy and his nephew save five Chin...,Not Rated,"Action, Adventure, Drama",7.7,6454,https://m.media-amazon.com/images/M/MV5BMjNkZT...
7081,The Wild Wild West,/title/tt0058855/,(1965–1969),"Two Secret Service agents, equipped with a wid...",TV-PG,"Action, Adventure, Western",8.1,5706,https://m.media-amazon.com/images/M/MV5BOTAxMT...


In [334]:
titles_df.to_csv('./data/imdb_titles_df.csv', index=False)

In [265]:
no_dupes = titles_df.drop_duplicates(subset='href')
no_dupes.shape

(2484, 9)

In [333]:
no_dupes.to_csv('./data/imdb_titles_no_dupes.csv', index=False)